## Importing the necessary libraries

In [1]:
from bs4 import BeautifulSoup as bs # for manipulating xml files
import os # for manipulating files and directories in the operating system
import pandas as pd # for manipulating dataframes

## Create a new dataframe

In [12]:
df = pd.DataFrame(
{"Text" :  [ ],"Filler" :  [ ],
 "Repetition" :  [ ],"Japanese_Word" :  [ ],
 "Unfinished_Sentence" :  [ ],"Self-Correction" :  [ ],
 "Short_Pause" :  [ ],"Long_Pause" :  [ ], "SST_LEVEL" :  [  ]})


## Converting the files into a single csv dataframe


In [13]:
for filename in os.listdir(r"DatasetModifiedTags"): # loop through each of the xml files inside the directory: DatasetModifiedTags
   with open(os.path.join(r"DatasetModifiedTags", filename), 'r') as f: 

       content = []
       content = f.readlines()
       content = "".join(content)
       bsc = bs(content, "lxml") #using the beautifulsoup library to read the content of each file

       f.close()
        
       #remove unnecessary interviewer speech
        
       interviewer = bsc.find_all("a")
       for item in interviewer:
            item.decompose()

    
       
        
         #get some useful features from the learner speech
       filler_number = len(bsc.find_all("f"))      #get the number of <f> tags (filled pause)
       repetition_number = len(bsc.find_all("r"))  #get the number of <r> tags (repetition)
       jp_word_number = len(bsc.find_all("jp"))    #get the number of <jp> tags (use of japanese word)
       spause_number = len(bsc.find_all("d"))      #get the number of <d> tags  (short pause)
       lpause_number = len(bsc.find_all("dd"))     #get the number of <dd> tags (long pause)
       co_number = len(bsc.find_all("co"))         #get the number of <co> tags (cut-off or unfinished sentence) 
       sc_number = len(bsc.find_all("sc"))         #get the number of <sc> tags (self-correction)

       
        
       #remove unnecessary tags in the raw text like repetitions, non verbal signs etc.. 
       removed_tags = bsc.find_all(
           ["nvs", "ctxt", "r", "rq", "sc", "scq","h", "co", "f"]) 
        
       for item in removed_tags:
           item.decompose()  # the decompose function destroys the object (tag + its content)
            
       
       learner = bsc.find_all("b")  # all the learner speech data is now in a list

       level = bsc.find_all("sst_level")  # the target data (sst english level)
     
       elements = [l.get_text() for l in learner]
       text = " ".join(elements)  # text variable now contains all the text in a string instead of a list
            
            #create a row to insert in the dataframe
       new_row = {"Text": text, "Filler": filler_number,
              "Repetition": repetition_number, "Japanese_Word": jp_word_number, "Unfinished_Sentence": co_number,
              "Self-Correction": sc_number, "Short_Pause": spause_number, "Long_Pause": lpause_number
              , "SST_LEVEL": level[0].get_text()} 
        

       df = df.append(new_row, ignore_index=True)

print(df)

df.to_csv("LearnerDataset.csv",index=False) #save the dataframe into a .csv file

                                                   Text  Filler  Repetition  \
0     My name is . Hello.  I'm fine, but I'm a littl...   188.0        68.0   
1      I'm . Nice to meet you. Yes, of course.  I li...   138.0        74.0   
2     My name is . O K. Please call me .  Yes. I wor...   105.0        35.0   
3     My name is . Thank you. Nice to meet you.  I l...   136.0        34.0   
4       my name is . Nice to meet you, too. . I work...   303.0        49.0   
...                                                 ...     ...         ...   
1276  O K. My name is . Nice to meet you.  fine. But...   117.0        17.0   
1277  . Good morning.  I'm . Sure.  I'm a little tir...   159.0        16.0   
1278  My name is . Call me . Yes. I'm fine. Bit nerv...   140.0        41.0   
1279   my name is . Sure. Fine, thank you. How about...   137.0         7.0   
1280  My name is . Yeah. O K. How are you? I live in...    99.0        45.0   

      Japanese_Word  Unfinished_Sentence  Self-Corr